In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
# pip install openai==0.28.0
# pip install pypdf
# pip install docarray==0.32.1
# pip install tiktoken
# pip install pydantic==1.10.13  

# Import module

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

# use OpenAI API 

In [3]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
embeddings = OpenAIEmbeddings()
loader = PyPDFLoader("./acp-23-10423-2023.pdf")

c:\Users\K1\anaconda3\envs\mylangchainproject\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
c:\Users\K1\anaconda3\envs\mylangchainproject\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


# Embedding

In [4]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

# RetrievalQA with similarity search

In [5]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(search_kwargs={"k": 2}),  # top 2 scores
    verbose=True,
    return_source_documents=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [6]:
import textwrap
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'] + f": page{source.metadata['page']}")

In [8]:
query = "Please list the effects of hygroscopic cloud seeding."
llm_response = qa(query)

c:\Users\K1\anaconda3\envs\mylangchainproject\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [11]:
process_llm_response(llm_response)

The effects of hygroscopic cloud seeding can include:

1. Increased cloud activation: Hygroscopic cloud seeding can enhance the activation of cloud droplets, leading
to the formation of more cloud particles.

2. Increased cloud condensation: Cloud condensation is the process by which water vapor condenses onto cloud
particles. Hygroscopic cloud seeding can promote this process, resulting in the growth of cloud droplets.

3. Enhanced precipitation: Hygroscopic cloud seeding can increase the formation of raindrops by promoting the
growth of cloud droplets. This can lead to enhanced precipitation in the seeded area.

4. Altered microphysical processes: Hygroscopic cloud seeding can affect various microphysical processes
within clouds, such as evaporation and coalescence of cloud droplets.

5. Localized effects: The effects of hygroscopic cloud seeding are typically localized to the seeded area.
This means that the increase in precipitation is concentrated in the targeted region.

It's imp

# If you want to debug

In [10]:
import langchain
langchain.debug = True
qa(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Please list the effects of hygroscopic cloud seeding."
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Please list the effects of hygroscopic cloud seeding.",
  "context": "for Effective Hygroscopic Cloud Seeding, J. Appl. Meteorol.\nClim., 49, 1548–1562, https://doi.org/10.1175/2010jamc2307.1,\n2010.\nRosenfeld, D., Sherwood, S., Wood, R., and Donner, L.: Climate ef-\nfects of aerosol-cloud interactions, Science, 343, 379–380, 2014.\nSegal, Y ., Khain, A., Pinsky, M., and Rosenfeld, D.: Effects of hy-\ngroscopic seeding on raindrop formation as seen from simula-\ntions using a 2000-bin spectral cloud parcel model, Atmos. Res.,\n71, 3–34, 2004.\nSeto, J., Tomine, K., Wakimizu, K., and Nishiyama, K.: Artiﬁ-\ncial clou

{'query': 'Please list the effects of hygroscopic cloud seeding.',
 'result': "The effects of hygroscopic cloud seeding can include:\n\n1. Increased cloud activation: Hygroscopic cloud seeding can enhance the activation of cloud droplets, leading to the formation of more cloud particles.\n\n2. Increased cloud condensation: Cloud condensation is the process by which water vapor condenses onto cloud particles. Hygroscopic cloud seeding can promote this process, resulting in the growth of cloud droplets.\n\n3. Enhanced precipitation: Hygroscopic cloud seeding can increase the formation of raindrops by promoting the growth of cloud droplets. This can lead to enhanced precipitation in the seeded area.\n\n4. Altered microphysical processes: Hygroscopic cloud seeding can affect various microphysical processes within clouds, such as evaporation and coalescence of cloud droplets.\n\n5. Localized effects: The effects of hygroscopic cloud seeding are typically localized to the seeded area. This m